### StableLM

In [1]:
! pip install langchain --upgrade
! pip install torch --upgrade
! pip install transformers --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.9/709.9 kB 3.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.148
    Uninstalling langchain-0.0.148:
      Successfully uninstalled langchain-0.0.148


In [2]:
import torch
import textwrap
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList

tokenizer = AutoTokenizer.from_pretrained("stabilityai/stablelm-tuned-alpha-7b")
model = AutoModelForCausalLM.from_pretrained("stabilityai/stablelm-tuned-alpha-7b")
model.half().cuda()

class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [50278, 50279, 50277, 1, 0]
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
def ask(query):
    prompt = f"<|USER|> {query} <|ASSISTANT|>"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    tokens = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.5,
        do_sample=True,
        stopping_criteria=StoppingCriteriaList([StopOnTokens()])
    )

    print(tokenizer.decode(tokens[0], skip_special_tokens=True))

In [4]:
while True:
    input_prompt = "Human: "
    query = input(input_prompt)

    if query.lower() == "bye":
        text = f"{input_prompt}: {query}"
        print(textwrap.fill(text, width=80))
        print("AI: Bye!")
        break

    ask(query)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 tell me about rambutan. describe its color, shape, texture and all physical properties. Rambutan is a tropical fruit that belongs to the Rubus genus, which is also known as the Rambutan tree. It has a round, oval or heart-shaped fruit, with a smooth and shiny skin that is usually green or brown in color. The flesh of the fruit is also green or brown in color and has a slightly soft texture.

Rambutan has a deep red color and a sweet, mild flavor. It is a good source of vitamin C, vitamin A, vitamin B6, potassium, and magnesium. It is also rich in antioxidants, including flavonoids and carotenoids.

The rambutan tree has a long history of use in traditional medicine. It is believed to have various medicinal properties, including anti-inflammatory, anti-anxiety, and anti-inflammatory effects. It is also used as a natural insecticide and for its ornamental value.
Human: : bye
AI: Bye!
